<a href="https://colab.research.google.com/github/LuisMHS/Python-ESP32WROVER/blob/main/Interfaz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import flet as ft
import cv2
import time
import base64
from threading import Thread
import os

class CameraStream:
    def __init__(self, url):
        self.cap = cv2.VideoCapture(url)
        self.running = True
        self.frame = None
        self.recording = False
        self.video_writer = None

    def start(self):
        def run():
            while self.running:
                ret, frame = self.cap.read()
                if ret:
                    self.frame = frame
                    if self.recording and self.video_writer:
                        self.video_writer.write(frame)
                time.sleep(0.03)
        Thread(target=run, daemon=True).start()

    def get_frame_base64(self):
        if self.frame is None:
            return None
        _, buffer = cv2.imencode('.jpg', self.frame)
        return base64.b64encode(buffer).decode('utf-8')

    def take_photo(self):
        if self.frame is not None:
            os.makedirs("ESP32_WROVER", exist_ok=True)
            filename = f"ESP32_WROVER/photo_{int(time.time())}.jpg"
            cv2.imwrite(filename, self.frame)
            return filename
        return None

    def start_recording(self):
        if self.frame is not None:
            os.makedirs("ESP32_WROVER", exist_ok=True)
            h, w = self.frame.shape[:2]
            filename = f"ESP32_WROVER/video_{int(time.time())}.mp4"
            self.video_writer = cv2.VideoWriter(
                filename, cv2.VideoWriter_fourcc(*'XVID'), 20, (w, h)
            )
            self.recording = True
            return filename
        return None

    def stop_recording(self):
        self.recording = False
        if self.video_writer:
            self.video_writer.release()
            self.video_writer = None

    def stop(self):
        self.running = False
        self.cap.release()
        self.stop_recording()


def main(page: ft.Page):
    page.window.width=600
    page.window.height=750
    page.title = "Cámara ESP32 + Grabación"
    page.scroll = "auto"

    cam = None
    img = ft.Image(src="", width=640, height=480)
    status = ft.Text("Estado: Desconectado")

    ip_input = ft.TextField(label="Dirección IP de la cámara", width=250)
    btn_connect = ft.ElevatedButton("🔌 Conectar", disabled=False)
    btn_desconectar = ft.ElevatedButton("❌ Desconectar", disabled=True)
    btn_foto = ft.ElevatedButton("📸 Tomar foto", disabled=True)
    btn_grabar = ft.ElevatedButton("🎥 Iniciar grabación", disabled=True)
    btn_parar = ft.ElevatedButton("⏹️ Detener grabación", disabled=True)

    def update_image():
        while True:
            if cam:
                frame_data = cam.get_frame_base64()
                if frame_data:
                    img.src_base64 = frame_data
                    page.update()
            time.sleep(0.05)

    def connect_camera(e):
        nonlocal cam
        url = f"http://{ip_input.value.strip()}:81/stream"
        cam = CameraStream(url)
        cam.start()
        status.value = f"Conectado a {url}"
        btn_foto.disabled = False
        btn_grabar.disabled = False
        btn_parar.disabled = False
        btn_connect.disabled = True
        btn_desconectar.disabled = False
        ip_input.disabled = True
        page.update()

    def disconnect_camera(e):
        nonlocal cam
        if cam:
            cam.stop()
            cam = None
        img.src_base64 = ""
        status.value = "Estado: Desconectado"
        ip_input.disabled = False
        btn_connect.disabled = False
        btn_foto.disabled = True
        btn_grabar.disabled = True
        btn_parar.disabled = True
        btn_desconectar.disabled = True
        page.update()

    def take_photo(e):
        filename = cam.take_photo()
        if filename:
            status.value = f"Foto guardada: {filename}"
            page.update()

    def start_recording(e):
        filename = cam.start_recording()
        if filename:
            status.value = f"Grabando: {filename}"
            page.update()

    def stop_recording(e):
        cam.stop_recording()
        status.value = "Grabación detenida"
        page.update()

    btn_connect.on_click = connect_camera
    btn_desconectar.on_click = disconnect_camera
    btn_foto.on_click = take_photo
    btn_grabar.on_click = start_recording
    btn_parar.on_click = stop_recording

    page.add(
        ft.Row([ip_input, btn_connect, btn_desconectar]),
        img,
        ft.Column([btn_foto, btn_grabar, btn_parar],spacing=10),
        status
    )

    Thread(target=update_image, daemon=True).start()

ft.app(target=main)


